In [2]:
from sqlalchemy import create_engine
import pandas as pd

In [3]:
###CONNECT TO THE DATABASES 
engine = create_engine('sqlite:///C:\\Users\\e333775\\Documents\\BA510\\final-project-team-mike-jose-trish\\docs\\csv_data\\CoursesDB.db')
engine1 = create_engine('sqlite:///C:\\Users\\e333775\\Documents\\BA510\\final-project-team-mike-jose-trish\\docs\\csv_data\\CourseDataWarehouse.db')

In [29]:
#list of tables in OPERATIONAL_DB
pd.read_sql('select * from courses',engine).head(1)
pd.read_sql('select * from meetings',engine).head(1)
pd.read_sql('select * from instructors',engine).head(1)
pd.read_sql('select * from locations',engine).head(1) 
pd.read_sql('select * from catalog',engine).head(1)
pd.read_sql('select * from course_titles',engine).head(1)

,course_id,course_title
0,1,Introduction to Financial Accounting


In [17]:
#list of tables in DIMENSIONAL_DB
pd.read_sql('select * from programs',engine1).head(1)
pd.read_sql('select * from locations',engine1).head(1)
pd.read_sql('select * from catalog_names',engine1).head(1)
pd.read_sql('select  * from meetings',engine1).head(1)
pd.read_sql('select * from course_usage',engine1).head(1)


,name,building,room
0,DSB 105,DSB,105


In [6]:
###CREATE THE PROGRAMS TABLE AND FILL IT WITH INFO FROM OPERATIONAL DATABASE
programs = pd.read_sql('select distinct program_code, program_name from catalog',engine)
pd.io.sql.execute("""create table programs (program_id Integer, program_code varchar(20), program_name varchar(20))""",engine1)

for row in programs.itertuples():
    pd.io.sql.execute("""INSERT INTO Programs (Program_ID, program_code, program_name) VALUES ({},'{}','{}')""".format(row[0],row[1],row[2]),engine1)

In [17]:
###CREATE THE LOCATIONS TABLE AND FILL IT WITH INFO FROM OPERATIONAL DATABASE
buildings = pd.read_sql('select * from locations',engine)
pd.io.sql.execute("""CREATE table locations (name TEXT, building TEXT, room Text)""",engine1)

for row in buildings.itertuples():
    pd.io.sql.execute("""INSERT INTO locations (name, building, room) VALUES ('{}','{}','{}')""".format(row[1],row[2],row[3]),engine1)

In [92]:
###CREATE THE CATALOG_NAMES TABLE AND FILL IT WITH "CLEANED" INFO FROM OPERATIONAL DATABASE 
###CLEANED = OPERATIONAL DATABASE HAD BOTH 2017-2018, AND 2018-2019 FILES READ IN. 
###A DISTINCT CALL WAS USED IN ORDER TO DELETE DUPLICATES WHICH COULD CAUSE EXTRA RECORDS
catalog_names = pd.read_sql('select distinct catalog_id, course_title, credits from catalog',engine)

pd.io.sql.execute("""create table if not exists catalog_names (catalog_id TEXT, course_title TEXT, credits TEXT)""",engine1)

for row in catalog_names.itertuples():
    pd.io.sql.execute("""INSERT INTO catalog_names (catalog_id, course_title, credits) VALUES ('{}','{}','{}')""".format(row[1],row[2],row[3]),engine1)

In [27]:
###CREATE THE MEETINGS TABLE AND FILL IT WITH "CLEANED" & "RELEVANT" INFO FROM OPERATIONAL DATABASE. 
###CLEANED = WE NOTICED IN THE OPERATIONAL MEETINGS DATABASE, THERE ARE MANY DUPLICATES LISTED. DISTINCT WAS USED TO DELETE DUPS.
###RELEVANT = THE HYPOTHETICAL DOWNSTREAM USER IN THIS APPLICATION ONLY WANTS TO WORRY ABOUT CLASSES FROM SPRING2019.
###FILTERED DOWN TO SPRING2019 FOR THE DIMENSIONALDATAWAREHOUSE

import time
meetings_lookup = pd.read_sql("""select distinct term, crn, location, day, start,end from meetings where term like 'Spring2019'""",engine)

pd.io.sql.execute("""CREATE TABLE meetings (term TEXT, crn TEXT, location TEXT, day TEXT)""",engine1)

count = 0 
for row in meetings_lookup.itertuples():
    if count % 10000==0:
        print(count)
    pd.io.sql.execute("""INSERT INTO meetings (term, crn, location, day) VALUES ('{}','{}','{}','{}')""".format(row[1],row[2],row[3],row[4]),engine1)
    count = count + 1

0
10000
20000


In [24]:
###EXAMPLE OF DUPLICATES IN OPERATIONAL DATABASE REFERENCED ABOVE
pd.read_sql("""select * from meetings where day = 'S'""",engine).sort_values('start',ascending=False).head(1)

,term,crn,location,day,start,end
1427,Spring2019,39005,DSB 2109A,S,2019-05-11 09:00:00,2019-05-11 17:00:00
3008,Spring2019,39005,DSB 2109A,S,2019-05-11 09:00:00,2019-05-11 17:00:00
1422,Spring2019,38153,DSB 105,S,2019-05-04 14:00:00,2019-05-04 16:00:00
3003,Spring2019,38153,DSB 105,S,2019-05-04 14:00:00,2019-05-04 16:00:00
2993,Spring2019,38152,DSB 105,S,2019-05-04 14:00:00,2019-05-04 16:00:00


In [28]:
###WRITE A QUERY FROM THE OPERATIONAL_DATABASE TO RETURN THE ROLLED UP INFORMATION WE'D LIKE TO INSERT INTO DIMENSIONAL DB
###IN THIS CASE THE ROLLED UP INFO IS THE SUM OF ACTUAL STUDENTS IN THE CLASS, AND COUNT OF SECTIONS
new_usage =  pd.read_sql("""
select a.catalog_id, b.course_title, b.program_name, a.term, sum(a.act) as total_students,count(a.crn) as count_of_sections from courses a, (select distinct program_code, program_name, catalog_id, course_title from catalog) b
where a.catalog_id = b.catalog_id 
group by a.catalog_id,b.course_title, b.program_name, a.term order by 5 desc""",engine)

new_usage.head(1)

,catalog_id,course_title,program_name,term,total_students,count_of_sections
0,EN 0012,Texts and Contexts II: Writing About Literature,English,Spring2015,960,52


In [188]:
###CREATE TABLE COURSE USAGE FILLING IT WITH ROLLED UP DATA FROM THE OPERATIONAL DATABASE
new_usage =  pd.read_sql("""
select a.catalog_id, b.course_title, b.program_name, a.term, sum(a.act) as total_students,count(a.crn) as count_of_sections from courses a, (select distinct program_code, program_name, catalog_id, course_title from catalog) b
where a.catalog_id = b.catalog_id 
group by a.catalog_id,b.course_title, b.program_name, a.term order by 5 desc""",engine)

pd.io.sql.execute("""CREATE table IF NOT EXISTS course_usage (catalog_id  text, course_title text, program_name text, term TEXT, total_students integer, count_of_sections integer)""",engine1)

for row in new_usage.itertuples():
    pd.io.sql.execute("""INSERT into course_usage (catalog_id,course_title,program_name, term, total_students, count_of_sections) values ('{}','{}','{}','{}',{},{})""".format(row[1],row[2],row[3],row[4],row[5],row[6]),engine1)

In [10]:
###TEST ENTITY INTEGRITY OF COURSE_USAGE FACT TABLE. THERE ARE NO CATALOG_ID/TERM DUPLICATES
pd.read_sql("""
                SELECT MAX(COUNT_OF_INSTANCES)
                FROM 
                 (
                 SELECT catalog_id,term, count(course_title) as count_of_instances
                 FROM course_usage 
                 GROUP BY catalog_id,term
                 )
                 
                 """,engine1)

,MAX(COUNT_OF_INSTANCES)
0,1


In [13]:
###DOMAIN INTEGRITY. SINCE CATALOG_ID & TERM TOGETHER ARE OUR UNIQUE IDENTIFIER WE NEED TO ENSURE THIS FIELD IS NEVER NULL###
pd.read_sql("""select * from course_usage where (catalog_id is null or term is null)""",engine1)

,catalog_id,course_title,program_name,term,total_students,count_of_sections


In [36]:
#NOW USE DIMENSIONAL DATABASE TO SHOW UTILITY
#WHAT IS THE MOST POPULAR PROGRAM IN THE UNIVERSITY IN SPRING 2019
pd.read_sql("""SELECT program_name, term, sum(total_students) as total_students 
                FROM course_usage 
                WHERE term = 'Spring2019' 
                GROUP BY program_name, term 
                ORDER BY 3 desc""",engine1).head()

,program_name,term,total_students
0,Nursing,Spring2019,2485
1,English,Spring2019,1464
2,Biology,Spring2019,1355
3,Mathematics,Spring2019,1108
4,Accounting,Spring2019,1006


In [37]:
#NOW USE DIMENSIONAL DATABASE TO SHOW UTILITY
#WHAT IS THE MOST POPULAR COURSE IN THE UNIVERSITY IN SPRING 2019
pd.read_sql("""SELECT a.catalog_id,b.course_title, a.term, sum(a.total_students) as total_students 
                FROM course_usage a, catalog_names b 
                WHERE a.catalog_id = b.catalog_id 
                AND a.term = 'Spring2019' 
                GROUP BY a.catalog_id,b.course_title,a.term 
                ORDER BY 4 desc""",engine1).head()

,catalog_id,course_title,term,total_students
0,EN 0012,Texts and Contexts II: Writing About Literature,Spring2019,931
1,EC 0012,Introduction to Macroeconomics,Spring2019,540
2,MA 0217,Accelerated Statistics,Spring2019,477
3,AC 0012,Introduction to Management Accounting,Spring2019,472
4,PH 0101,Introduction to Philosophy,Spring2019,446


In [55]:
##NOW USE DIMENSIONAL DATABASE TO SHOW UTILITY
##WHAT BUILDING HAS THE MOST SATURDAY CLASSES
pd.read_sql("""SELECT b.building,a.term, count(a.day) as sat_classes 
                FROM meetings a, locations b 
                WHERE a.location = b.name 
                AND a.day = 'S' 
                GROUP BY b.building,a.term 
                HAVING b.building <> 'nan'
                ORDER BY 3 desc""",engine1)

,building,term,sat_classes
0,DSB,Spring2019,56
1,NHS,Spring2019,16
2,CMC,Spring2019,12
3,CNS,Spring2019,8
4,MCA,Spring2019,6
